# 🏀 NBA Api

### Dependencies

In [1]:
!pip3 install nba_api
!pip3 install pandas

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.2
    Uninstalling requests-2.32.2:
      Successfully uninstalled requests-2.32.2



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Andreu Picornell\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Andreu Picornell\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import time
from datetime import datetime

import pandas as pd
from pandas import DataFrame

from nba_api.stats.endpoints import playergamelog, teamgamelog, playercareerstats, commonplayerinfo

# 🔧 Functions

## Data Download

### Get Player Name

In [3]:
def get_player_name(id):
    
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=id)

    # Extract the player's name from the response
    player_name = player_info.get_normalized_dict()['CommonPlayerInfo'][0]['DISPLAY_FIRST_LAST']
    player_name = player_name.replace(" ", "-")
    
    return player_name

### Get Teams Info

In [4]:
def get_teams_info():
    # Get dictionary fo team ID's
    from nba_api.stats.static import teams
    teams_dict = teams.get_teams()
    df = pd.DataFrame(teams_dict)
    return df

### Get Seasons Played

In [5]:
def get_seasons_played(player_id):

    career_stats = playercareerstats.PlayerCareerStats(player_id=player_id)
    career_data = career_stats.get_data_frames()[0]
    player_stats = career_data[['SEASON_ID']]

    year_start = player_stats['SEASON_ID'].iloc[0][:4]
    year_end = player_stats['SEASON_ID'].iloc[len(player_stats) - 1][:4]
    year_start, year_end

    return year_start, year_end

### Get Player Stats Per Game

In [6]:
def get_player_stats_per_game(player_id, year_start, year_end):
    
    player_game_stats = pd.DataFrame()
    print('Downloading... ', end='')
    
    for year in range(year_start, year_end + 1):
        game_log = playergamelog.PlayerGameLog(player_id=player_id, season=year)
        game_data = game_log.get_data_frames()[0]
        player_game_stats = pd.concat([player_game_stats, game_data], ignore_index=True)
        print(f'{year}', end=' ')
        time.sleep(0.5)

    return player_game_stats

### Get Team Stats

In [7]:
def get_team_stats(team_id, year_start, year_end):
    team_id += 1610612700
    
    for year in range(year_start, year_end):
        season_stats = teamgamelog.TeamGameLog(team_id=team_id, season=str(year))
    
    # Convert to DataFrame
    season_stats_df = season_stats.get_data_frames()[0]  # Extract the first dataframe
    
    return season_stats_df

In [8]:
def get_team_stats(team_id, year_start, year_end):

    team_stats = pd.DataFrame()
    
    for year in range(year_start, year_end):
        season_stats = teamgamelog.TeamGameLog(team_id=team_id, season=str(year))
                
        team_stats = pd.concat([team_stats, season_stats], ignore_index=True)
        print(f'Finished {year}')
        time.sleep(0.2)
    
    team_stats = teamgamelog.TeamGameLog(team_id=team_id, season=season)
    stats = team_stats.get_data_frames()[0]
    
    return stats

## Transformations

### Get Opponent ID

In [9]:
def get_opponent_id(df, teams_df):

    df['MATCHUP'] = df['MATCHUP'].apply(lambda x: x[-3:])

    # Create a lookup dictionary from teams_df
    team_id_lookup = teams_df.set_index('abbreviation')['id'].to_dict()
    
    # Keep only the last 2 digits of each ID
    team_id_lookup = {key: str(value)[-2:] for key, value in team_id_lookup.items()}

    # Optionally, you can convert the sliced string back to an integer if needed
    team_id_lookup = {key: value for key, value in team_id_lookup.items()}

    # Map the abbreviations in player_all_games['MATCHUP'] to their respective team IDs
    df['OPPONENT_ID'] = df['MATCHUP'].map(team_id_lookup)

    return df

### Season ID

In [10]:
def format_season_id(df):
    df['SEASON_ID'] = df['SEASON_ID'].astype(str).str[1:]
    return df

### Date To Weekday

In [11]:
def date_to_weekday(date_str):

    date_object = datetime.strptime(date_str, "%b %d, %Y")
    return date_object.isoweekday()

### Date To Year Day

In [12]:
def date_to_year_day(date_str):
    
    date_object = datetime.strptime(date_str, "%b %d, %Y")
    day_of_year = date_object.timetuple().tm_yday
    
    return day_of_year

### Encode Win

In [13]:
def encode_win(df):
    df['W'] = df['WL'].map({'W': 1, 'L': 0}).astype('int')
    return df

## Feature Extraction

### Calculate Rest Days

In [14]:
def calculate_rest_days(df, date_column='GAME_DATE'):
    
    data = df.copy()
    # Ensure the date column is in datetime format
    data[date_column] = pd.to_datetime(data[date_column], format="%b %d, %Y")
    
    # Sort the dataframe by date to ensure correct rest day calculation
    data = data.sort_values(by=date_column).reset_index(drop=True)

    # Calculate time since last game using sorted dates
    data['REST_DAYS'] = (data[date_column] - data[date_column].shift(1)).dt.total_seconds() / (24 * 60 * 60)
    
    # Fill NaN values with 0 (first game has no previous game)
    data['REST_DAYS'] = data['REST_DAYS'].fillna(0).astype(int)
    
    return data


### Calculate Win Percentages

In [15]:
def calculate_win_percentages(df):
    # Initialize columns for total win percentage and opponent win percentage
    df['W%_TOTAL'] = 0.0
    df['W%_OPPONENT'] = 0.0

    # Dictionaries to keep track of totals
    total_games = 0
    total_wins = 0
    opponent_stats = {}

    # Iterate through rows to calculate cumulative win percentages
    for index, row in df.iterrows():
        opponent_id = row['OPPONENT_ID']
        result = row['W']  # 'W' column indicates win ('1') or loss ('0')

        # Update total stats
        total_games += 1
        if result == 1:
            total_wins += 1
        
        # Update opponent-specific stats
        if opponent_id not in opponent_stats:
            opponent_stats[opponent_id] = {'games': 0, 'wins': 0}
        opponent_stats[opponent_id]['games'] += 1
        if result == 1:
            opponent_stats[opponent_id]['wins'] += 1
        
        # Calculate win percentages
        df.at[index, 'W%_TOTAL'] = round(total_wins / total_games, 2)
        df.at[index, 'W%_OPPONENT'] = round((
            opponent_stats[opponent_id]['wins'] / opponent_stats[opponent_id]['games']
        ), 2)

    return df

### Calculate Recent Win Percentage

In [16]:
def calculate_recent_win_percentage(df, n=5):
    """
    Calculates W%_RECENT for the n most recent games.

    Parameters:
        df (pd.DataFrame): Input DataFrame containing game data.
        n (int): Number of recent games to consider for win percentage.

    Returns:
        pd.DataFrame: DataFrame with a new column 'W%_RECENT'.
    """
    # Ensure 'W' column is integer type
    df['W'] = df['W'].astype(int)

    # Initialize the new column
    df['W%_RECENT'] = 0.0

    # List to act as a sliding window for the most recent n games
    recent_results = []

    for index, row in df.iterrows():
        # Add current game's result to the recent_results list
        recent_results.append(row['W'])

        # Keep only the most recent n results
        if len(recent_results) > n:
            recent_results.pop(0)

        # Calculate win percentage for recent games
        recent_wins = sum(recent_results)
        recent_games = len(recent_results)

        # Update the column for the current row
        df.at[index, 'W%_RECENT'] = round(recent_wins / recent_games, 2)

    return df


### Calculate PPG vs Opponent

In [17]:
def calculate_ppg_vs_opponent(df):
    """
    Calculates and updates the 'PPG_VS_OPPONENT' column for each game in the season.

    Parameters:
        df (pd.DataFrame): Player's game logs with columns 'OPPONENT_TEAM_ID' and 'PTS'.

    Returns:
        pd.DataFrame: DataFrame with updated 'PPG_VS_OPPONENT' column.
    """
    # Initialize the new column for PPG against the opponent
    df['PPG_VS_OPPONENT'] = 0.0
    
    # Dictionary to keep track of the cumulative points and games for each opponent
    opponent_stats = {}

    for index, row in df.iterrows():
        opponent_team_id = row['OPPONENT_ID']
        points_scored = row['PTS']
        
        # Update stats for the opponent
        if opponent_team_id not in opponent_stats:
            opponent_stats[opponent_team_id] = {'games': 0, 'points': 0}

        # Add current game points
        opponent_stats[opponent_team_id]['games'] += 1
        opponent_stats[opponent_team_id]['points'] += points_scored
        
        # Calculate PPG against the opponent
        avg_ppg_vs_opponent = opponent_stats[opponent_team_id]['points'] / opponent_stats[opponent_team_id]['games']
        
        # Update the PPG_VS_OPPONENT column for the current game
        df.at[index, 'PPG_VS_OPPONENT'] = round(avg_ppg_vs_opponent, 2)

    return df


## New Data

### Create DF With All Stats

In [18]:
def create_df_all_stats(player, teams_df):
    year_start, year_end = get_seasons_played(player)
    player_all_games = get_player_stats_per_game(player, int(year_start), int(year_end))

    player_all_games['YEAR_DAY'] = player_all_games['GAME_DATE'].apply(date_to_year_day)
    player_all_games['WEEK_DAY'] = player_all_games['GAME_DATE'].apply(date_to_weekday)

    player_all_games = calculate_rest_days(player_all_games)
    get_opponent_id(player_all_games, teams_df)
    format_season_id(player_all_games)
    encode_win(player_all_games)
    calculate_win_percentages(player_all_games)
    calculate_recent_win_percentage(player_all_games, 10)
    calculate_ppg_vs_opponent(player_all_games)


    player_all_games = player_all_games.drop(columns=["Player_ID", "WL", "FGA", "FG3A", "FTA", "REB", "Game_ID", "GAME_DATE", "MATCHUP", "VIDEO_AVAILABLE"])
    return player_all_games

### Export CSV

In [19]:
def export_csv(df, id):
    player_name = get_player_name(id)
    date_formatted = datetime.now().strftime("%d%m%Y")
    
    df.to_csv(f'{player_name}-{date_formatted}.csv', index=False)

## Pending

In [20]:
import pandas as pd
from nba_api.stats.endpoints import playergamelog, teamgamelog, playercareerstats
from nba_api.stats.static import teams

def get_player_game_stats(player_id):
    """
    Get historical game stats for a player.
    player_id: int, the player's unique identifier (LeBron James is 2544).
    
    Returns a DataFrame with LeBron's game stats.
    """
    game_log = playergamelog.PlayerGameLog(player_id=player_id)
    game_data = game_log.get_data_frames()[0]
    return game_data

def get_team_stats(team_id, season="2024"):
    """
    Get team stats (offensive and defensive ratings) for a given season.
    team_id: int, the team's unique identifier.
    season: str, the season (e.g., "2024").
    
    Returns a dictionary with team statistics.
    """
    team_stats = teamgamelog.TeamGameLog(team_id=team_id, season=season)
    stats = team_stats.get_data_frames()[0]
    
    # team_stats_dict = {
    #     'offensive_rating': stats['OFF_RATING'].mean(),
    #     'defensive_rating': stats['DEF_RATING'].mean(),
    #     'win_percentage': stats['WL'].value_counts(normalize=True).get('W', 0)
    # }
    
    # return team_stats_dict
    return stats

def get_opponent_stats(opponent_team_id, season="2024"):
    """
    Get opponent team stats for a given season.
    opponent_team_id: int, the opponent team's unique identifier.
    season: str, the season (e.g., "2024").
    
    Returns a dictionary with opponent team statistics.
    """
    return get_team_stats(opponent_team_id, season)

def get_player_avg_ppg_against_opponent(player_id, opponent_team_id):
    """
    Calculate the average points per game LeBron has scored against a specific opponent.
    player_id: int, LeBron's player ID.
    opponent_team_id: int, the opponent team's ID.
    
    Returns the average points per game against that opponent.
    """
    game_log = get_player_game_stats(player_id)
    opponent_games = game_log[game_log['TEAM_ID'] == opponent_team_id]
    avg_ppg = opponent_games['PTS'].mean() if len(opponent_games) > 0 else 0
    return avg_ppg

def get_game_features(player_id, game_data, season="2024"):
    """
    Get the features for a specific game that LeBron James played.
    player_id: int, LeBron's player ID.
    game_data: DataFrame row with game information.
    season: str, the season (e.g., "2024").
    
    Returns a dictionary with the game features.
    """
    game_id = game_data['GAME_ID']
    team_id = game_data['TEAM_ID']
    opponent_team_id = game_data['OPPONENT_ID']
    
    # Features for the game
    features = {
        'label': game_data['PTS'],  # Points scored by LeBron in the game
        'avg_ppg_against_opponent': get_player_avg_ppg_against_opponent(player_id, opponent_team_id),
        'avg_ppg_current_season': get_player_avg_ppg_current_season(player_id, season),
        'avg_ppg_historical': get_player_avg_ppg_historical(player_id),
        'team_offensive_rating': get_team_stats(team_id, season)['offensive_rating'],
        'team_defensive_rating': get_team_stats(team_id, season)['defensive_rating'],
        'team_win_percentage': get_team_stats(team_id, season)['win_percentage'],
        'team_win_percentage_home': get_team_win_percentage_home(team_id, season),
        'team_win_percentage_against_opponent': get_team_win_percentage_against_opponent(team_id, opponent_team_id, season),
        'opponent_defensive_rating': get_opponent_stats(opponent_team_id, season)['defensive_rating'],
        'opponent_win_percentage': get_opponent_stats(opponent_team_id, season)['win_percentage'],
        'opponent_win_percentage_home': get_opponent_win_percentage_home(opponent_team_id, season),
        'home_game': 1 if game_data['GAME_LOCATION'] == 'home' else 0  # Home game or away game
    }
    
    return features

def get_player_avg_ppg_current_season(player_id, season="2024"):
    """
    Get the average points per game for LeBron James in the current season.
    player_id: int, LeBron's player ID.
    season: str, the season (e.g., "2024").
    
    Returns the average points per game for the current season.
    """
    game_log = get_player_game_stats(player_id)
    season_games = game_log[game_log['SEASON_ID'] == season]
    avg_ppg = season_games['PTS'].mean() if len(season_games) > 0 else 0
    return avg_ppg

def get_player_avg_ppg_historical(player_id):
    """
    Get the historical average points per game for LeBron James.
    player_id: int, LeBron's player ID.
    
    Returns the historical average points per game.
    """
    career_stats = playercareerstats.PlayerCareerStats(player_id=player_id)
    stats = career_stats.get_data_frames()[0]
    avg_ppg = stats['PTS'].mean() if len(stats) > 0 else 0
    return avg_ppg

def get_team_win_percentage_home(team_id, season="2024"):
    """
    Get the team's win percentage at home during the current season.
    team_id: int, the team's unique identifier.
    season: str, the season (e.g., "2024").
    
    Returns the team's win percentage at home.
    """
    team_stats = teamgamelog.TeamGameLog(team_id=team_id, season=season)
    stats = team_stats.get_data_frames()[0]
    home_games = stats[stats['MATCHUP'].str.contains('vs')]
    win_percentage = home_games['WL'].value_counts(normalize=True).get('W', 0)
    return win_percentage

def get_team_win_percentage_against_opponent(team_id, opponent_team_id, season="2024"):
    """
    Get the team's win percentage against a specific opponent.
    team_id: int, the team's unique identifier.
    opponent_team_id: int, the opponent team's unique identifier.
    season: str, the season (e.g., "2024").
    
    Returns the team's win percentage against the opponent.
    """
    team_stats = teamgamelog.TeamGameLog(team_id=team_id, season=season)
    stats = team_stats.get_data_frames()[0]
    opponent_games = stats[stats['OPPONENT_ID'] == opponent_team_id]
    win_percentage = opponent_games['WL'].value_counts(normalize=True).get('W', 0)
    return win_percentage

def build_lebron_features(player_id):
    """
    Build a DataFrame with all the games LeBron James has ever played.
    player_id: int, LeBron's player ID.
    
    Returns a DataFrame with features for every game.
    """
    game_data = get_player_game_stats(player_id)
    
    # Prepare the features for each game
    all_game_features = []
    for _, game in game_data.iterrows():
        features = get_game_features(player_id, game)
        all_game_features.append(features)
    
    # Convert to DataFrame
    df = pd.DataFrame(all_game_features)
    return df


# 📋 Teams

In [21]:
teams_df = get_teams_info()
teams_df = teams_df[['id', 'abbreviation']]
teams_df

,id,abbreviation
0,1610612737,ATL
1,1610612738,BOS
2,1610612739,CLE
3,1610612740,NOP
4,1610612741,CHI
5,1610612742,DAL
6,1610612743,DEN
7,1610612744,GSW
8,1610612745,HOU
9,1610612746,LAC


In [22]:
new_row = {'id': 1610612767, 'abbreviation': 'NJN'}

# Add the new row using pd.concat
teams_df = pd.concat([teams_df, pd.DataFrame([new_row])], ignore_index=True)

In [23]:
new_row = {'id': 1610612768, 'abbreviation': 'NOH'}

# Add the new row using pd.concat
teams_df = pd.concat([teams_df, pd.DataFrame([new_row])], ignore_index=True)

In [24]:
new_row = {'id': 1610612769, 'abbreviation': 'SEA'}

# Add the new row using pd.concat
teams_df = pd.concat([teams_df, pd.DataFrame([new_row])], ignore_index=True)

In [25]:
new_row = {'id': 1610612770, 'abbreviation': 'NOK'}

# Add the new row using pd.concat
teams_df = pd.concat([teams_df, pd.DataFrame([new_row])], ignore_index=True)

In [26]:
new_row = {'id': 1610612771, 'abbreviation': 'PHO'}

# Add the new row using pd.concat
teams_df = pd.concat([teams_df, pd.DataFrame([new_row])], ignore_index=True)

In [27]:
new_row = {'id': 1610612772, 'abbreviation': 'GOS'}

# Add the new row using pd.concat
teams_df = pd.concat([teams_df, pd.DataFrame([new_row])], ignore_index=True)

# 👑 Player

In [28]:
player = 2544

In [29]:
player_all_games = create_df_all_stats(player, teams_df)
player_all_games

Downloading... 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 

,SEASON_ID,MIN,FGM,FG_PCT,FG3M,FG3_PCT,FTM,FT_PCT,OREB,DREB,...,PLUS_MINUS,YEAR_DAY,WEEK_DAY,REST_DAYS,OPPONENT_ID,W,W%_TOTAL,W%_OPPONENT,W%_RECENT,PPG_VS_OPPONENT
0,2003,42,12,0.600,0,0.000,1,0.333,2,4,...,-9,302,3,0,58,0,0.00,0.00,0.0,25.00
1,2003,41,8,0.471,1,0.200,4,0.571,2,10,...,-3,303,4,1,56,0,0.00,0.00,0.0,21.00
2,2003,39,3,0.250,0,0.000,2,1.000,0,4,...,-21,305,6,2,57,0,0.00,0.00,0.0,8.00
3,2003,41,3,0.273,0,0.000,1,1.000,2,9,...,-3,309,3,4,43,0,0.00,0.00,0.0,7.00
4,2003,44,8,0.444,1,0.500,6,0.857,0,5,...,-7,311,5,2,54,0,0.00,0.00,0.0,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522,2024,30,13,0.650,2,0.400,2,1.000,0,3,...,4,3,5,1,37,1,0.65,0.71,0.6,27.59
1523,2024,36,8,0.500,4,0.571,1,0.500,1,12,...,-13,5,7,2,45,0,0.65,0.51,0.6,25.31
1524,2024,35,6,0.500,1,0.250,5,0.833,0,10,...,-12,7,2,2,42,0,0.64,0.59,0.6,26.95
1525,2024,32,7,0.636,2,0.667,2,0.500,1,4,...,-15,13,1,6,59,0,0.64,0.54,0.5,27.11


In [30]:
player_all_games.describe()

,MIN,FGM,FG_PCT,FG3M,FG3_PCT,FTM,FT_PCT,OREB,DREB,AST,...,PTS,PLUS_MINUS,YEAR_DAY,WEEK_DAY,REST_DAYS,W,W%_TOTAL,W%_OPPONENT,W%_RECENT,PPG_VS_OPPONENT
count,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,...,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000,1527.000000
mean,37.872299,9.924689,0.507375,1.629339,0.309884,5.568435,0.723026,1.151277,6.345776,7.411919,...,27.047151,4.795678,165.409299,4.083824,5.074656,0.644401,0.599398,0.603137,0.643242,26.596411
std,5.207874,3.081073,0.110423,1.520546,0.243382,3.335794,0.209088,1.163573,2.749411,2.969471,...,7.808681,12.823335,139.440586,1.924061,22.961525,0.478851,0.094122,0.181860,0.186719,3.111487
min,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,-39.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
25%,35.000000,8.000000,0.438000,0.000000,0.000000,3.000000,0.600000,0.000000,4.000000,5.000000,...,22.000000,-4.000000,42.000000,2.000000,2.000000,0.000000,0.550000,0.520000,0.500000,25.625000
50%,38.000000,10.000000,0.500000,1.000000,0.333000,5.000000,0.750000,1.000000,6.000000,7.000000,...,27.000000,6.000000,88.000000,4.000000,2.000000,1.000000,0.650000,0.620000,0.700000,27.250000
75%,41.000000,12.000000,0.579000,2.000000,0.500000,8.000000,0.857000,2.000000,8.000000,9.000000,...,32.000000,14.000000,324.000000,6.000000,3.000000,1.000000,0.660000,0.710000,0.800000,28.250000
max,55.000000,23.000000,0.929000,9.000000,1.000000,24.000000,1.000000,7.000000,19.000000,19.000000,...,61.000000,39.000000,366.000000,7.000000,258.000000,1.000000,0.670000,1.000000,1.000000,38.000000


In [31]:
player_all_games.columns

Index(['SEASON_ID', 'MIN', 'FGM', 'FG_PCT', 'FG3M', 'FG3_PCT', 'FTM', 'FT_PCT',
       'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
       'YEAR_DAY', 'WEEK_DAY', 'REST_DAYS', 'OPPONENT_ID', 'W', 'W%_TOTAL',
       'W%_OPPONENT', 'W%_RECENT', 'PPG_VS_OPPONENT'],
      dtype='object')

### Backup

In [32]:
player_all_games_backup = player_all_games.copy()
player_all_games_backup

,SEASON_ID,MIN,FGM,FG_PCT,FG3M,FG3_PCT,FTM,FT_PCT,OREB,DREB,...,PLUS_MINUS,YEAR_DAY,WEEK_DAY,REST_DAYS,OPPONENT_ID,W,W%_TOTAL,W%_OPPONENT,W%_RECENT,PPG_VS_OPPONENT
0,2003,42,12,0.600,0,0.000,1,0.333,2,4,...,-9,302,3,0,58,0,0.00,0.00,0.0,25.00
1,2003,41,8,0.471,1,0.200,4,0.571,2,10,...,-3,303,4,1,56,0,0.00,0.00,0.0,21.00
2,2003,39,3,0.250,0,0.000,2,1.000,0,4,...,-21,305,6,2,57,0,0.00,0.00,0.0,8.00
3,2003,41,3,0.273,0,0.000,1,1.000,2,9,...,-3,309,3,4,43,0,0.00,0.00,0.0,7.00
4,2003,44,8,0.444,1,0.500,6,0.857,0,5,...,-7,311,5,2,54,0,0.00,0.00,0.0,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522,2024,30,13,0.650,2,0.400,2,1.000,0,3,...,4,3,5,1,37,1,0.65,0.71,0.6,27.59
1523,2024,36,8,0.500,4,0.571,1,0.500,1,12,...,-13,5,7,2,45,0,0.65,0.51,0.6,25.31
1524,2024,35,6,0.500,1,0.250,5,0.833,0,10,...,-12,7,2,2,42,0,0.64,0.59,0.6,26.95
1525,2024,32,7,0.636,2,0.667,2,0.500,1,4,...,-15,13,1,6,59,0,0.64,0.54,0.5,27.11


### Export

In [33]:
export_csv(player_all_games_backup, player)